In [16]:
import os
os.environ["OMP_NUM_THREADS"] = "32"
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl
import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

In [17]:
# Loading blocks...
def load_blocks(blocks):
    with open (blocks, "rb") as fh:
        bs = dill.load(fh)[0:6]
    return bs

def filterByFDR(g, level, pval, keepOnlyMain=False):
    # Filtering edges
    pvals = np.array(g.edge_properties[pval].a)

    fdr_ep = g.new_ep("bool", True)
    fdr_ep.a = lsu(pvals, q=level)

    tv = GraphView(g, efilt=fdr_ep)

    # Keeping largest component
    if keepOnlyMain:
        comp, hist = label_components(tv)
        main_component = tv.new_vp("bool", (comp.a == np.where(hist == max(hist))[0][0]))
        tv.vertex_properties["main_component"] = main_component
        tv.set_vertex_filter(main_component)
    return tv

def filterBySign(g, ep, positive = True, keepOnlyMain = False):
    # Filtering edges
    corr = g.edge_properties[ep]
    sign = g.new_ep("bool", True)
    if positive:
        sign.a = np.array(corr.a > 0)
    else:
        sign.a = np.array(corr.a < 0)    

    tv = GraphView(g, efilt=sign)

    # Keeping largest component
    if keepOnlyMain:
        comp, hist = label_components(tv)
        main_component = tv.new_vp("bool", (comp.a == np.where(hist == max(hist))[0][0]))
        tv.vertex_properties["main_component"] = main_component
        tv.set_vertex_filter(main_component)
    return tv
    
# For each unique element in x, find its first apperance in x
def first_occurrence(x):
    _, idx = np.unique(x, return_index=True)
    return idx

def labelVertices(state):
    g = state.g
    g.vp.level_0 = g.new_vertex_property("double", state.get_bs()[0])
    first = first_occurrence(np.array([g.vp.level_0.a]))
    state.g.vp.labels = g.new_vp("string", [str(int(x)) if i in first else "" for i, x in enumerate(g.vp.level_0.a)])
    return state

In [18]:
g_path = '../../SBM/snakemake-layer/cache/trimmed_graph/fdr-1e-3/layered/'
tissues = ['head', 'body']
conditions = ['hs', 'ctrl']
graphs = {f'{tissue}':load_graph(g_path + f'{tissue}.xml.gz') for tissue in tissues}
b_path = '../../SBM/snakemake-layer/cache/MCMC/blocks/fdr-1e-3/layered/'
blocks = {f'{tissue}':load_blocks(b_path + f'{tissue}.dill') for tissue in tissues}

labels = [f'{tissue}-{condition}' for tissue in tissues for condition in conditions]

# Read clip graphs
clip_path = '../../cache/'
clip_graphs = {f'{tissue}':load_graph(clip_path + f'clip_g_{tissue}.xml.gz') for tissue in tissues}

In [19]:
cond_graphs = {}
for t in tissues:
    g = graphs[t]
    ds = g.ep.dataset
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        u = GraphView(g, efilt=lambda e: ds[e] == c)
        u = Graph(u, prune=True)  
        cond_graphs[l] = u  

head-hs
head-ctrl
body-hs
body-ctrl


In [20]:
cond_graphs

{'head-hs': <Graph object, undirected, with 6826 vertices and 4767735 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7fe40dd424d0>,
 'head-ctrl': <Graph object, undirected, with 6826 vertices and 5097528 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7fe40f187d50>,
 'body-hs': <Graph object, undirected, with 6575 vertices and 1303683 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7fe39c327bd0>,
 'body-ctrl': <Graph object, undirected, with 6575 vertices and 2924794 edges, 2 internal vertex properties, 5 internal edge properties, at 0x7fe3ee26f4d0>}

In [21]:
state = {}
for t in tissues:
    for c in conditions:
        l = f'{t}-{c}'
        print(l)
        g = cond_graphs[l]
        bs = blocks[t]
        # Arctan transform on correlations...
        corr = g.edge_properties["spearman"]
        g.ep.weight = g.new_edge_property("double", (2*np.arctanh(corr.a)))

        # Creating nested block model...
        state[l] = NestedBlockState(g, bs=bs,
                                    state_args=dict(recs=[g.ep.weight],
                                                    rec_types=["real-normal"]))

head-hs
head-ctrl
body-hs
body-ctrl


In [22]:
pos = {}
for t in tissues:
    c = 'ctrl'
    l = f'{t}-{c}'
    s = state[l]
    g = cond_graphs[l]
    labelVertices(s)
    print(l)
    pos_t, twat, tpos = s.draw( eorder=s.g.ep.weight,
            # edge_pen_width = 0,
            edge_color=prop_to_size(s.g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 10,
            vertex_text = s.g.vp.labels,
            vertex_text_position='centered',
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = s.g.vp.level_0,
            vertex_fill_color = s.g.vp.level_0,
            subsample_edges = 30000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000))
    pos[t] = pos_t
for t in tissues:
    c = 'hs'
    l = f'{t}-{c}'
    s = state[l]
    g = cond_graphs[l]
    g.vp.pos = g.copy_property(pos[t])
    labelVertices(s)
    print(l)
    s.draw( eorder=s.g.ep.weight,
            # edge_pen_width = 0,
            pos = g.vp.pos, 
            edge_color=prop_to_size(s.g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 10,
            vertex_text = s.g.vp.labels,
            vertex_text_position='centered',
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = s.g.vp.level_0,
            vertex_fill_color = s.g.vp.level_0,
            subsample_edges = 30000,
            output = "../../tmp/" + l + ".png", 
            output_size=(2000, 2000))  

head-ctrl
body-ctrl
head-hs
body-hs


In [29]:
def makeClipGraph (current_tissue, clip_fdr, pos, n_edges = 50000):

    # output path
    output_path = "../../tmp/clip/" + "fdr-" + str(clip_fdr) + "/" + current_tissue + "-nEdges_" + str(n_edges) 
    # create output directory
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    g_clip = clip_graphs[current_tissue]
    gNeg= filterByFDR(g_clip, clip_fdr, 'clip_p')
    gNeg = Graph(gNeg, prune=True)

    gNeg = filterBySign(gNeg, 'clip_c', positive = False)
    gNeg = Graph(gNeg, prune=True)
    gNeg.ep.clip_c.a = np.abs(gNeg.ep.clip_c.a)

    bs = blocks[current_tissue]

    state = NestedBlockState(gNeg, bs=bs,
                            state_args=dict(recs=[gNeg.ep.clip_c],
                                            rec_types=["real-normal"]))
    labelVertices(state)
    gNeg.vp.pos = gNeg.copy_property(pos)
    state.draw(eorder=gNeg.ep.clip_c,
                pos = gNeg.vp.pos, 
                edge_color=prop_to_size(gNeg.ep.clip_c, mi=0, ma=1, power=1, log=False),
                ecmap=(mpl.inferno, .6), 
                edge_gradient=[], 
                vertex_size = 10,
                vertex_text = state.g.vp.labels,
                vertex_text_position='centered',
                hvertex_size = 25,
                hedge_pen_width = 3,
                subsample_edges = n_edges,
                output = output_path + "-decohere.png", 
                output_size=(2000, 2000))

    g_clip = clip_graphs[current_tissue]
    gPos = filterByFDR(g_clip, clip_fdr, 'clip_p')
    gPos = Graph(gPos, prune=True)
    gPos = filterBySign(gPos, 'clip_c', positive = True)
    gPos = Graph(gPos, prune=True)
    gPos.vp.pos = gPos.copy_property(pos)

    bs = blocks[current_tissue]
    state = NestedBlockState(gPos, bs=bs,
                            state_args=dict(recs=[gPos.ep.clip_c],
                                            rec_types=["real-normal"]))
    labelVertices(state)
    state.draw( eorder=gPos.ep.clip_c,
                pos = gPos.vp.pos, 
                edge_color=prop_to_size(gPos.ep.clip_c, mi=0, ma=1, power=1, log=False),
                ecmap=(mpl.inferno, .6), 
                edge_gradient=[], 
                vertex_size = 10,
                vertex_text = state.g.vp.labels,
                vertex_text_position='centered',
                hvertex_size = 25,
                hedge_pen_width = 3,
                subsample_edges = n_edges,
                output = output_path + "-integrate.png", 
                output_size=(2000, 2000))
    return gNeg, gPos, state


In [30]:
# Make clip graphs for various FDRs
fdrs = [1e-1, 1e-2, 1e-3, 1e-4]
n_edges = 50000
for fdr in fdrs:
    print(f"Making clip graphs for FDR {fdr}")
    for tissue in tissues:
        print(f"Tissue: {tissue}")
        makeClipGraph(tissue, fdr, pos[tissue], n_edges)

Making clip graphs for FDR 0.1
Tissue: head
Tissue: body
Making clip graphs for FDR 0.01
Tissue: head
Tissue: body
Making clip graphs for FDR 0.001
Tissue: head
Tissue: body
Making clip graphs for FDR 0.0001
Tissue: head
Tissue: body


In [46]:
g_clip = clip_graphs['head']
clip_fdr = 0.001
gNeg= filterByFDR(g_clip, clip_fdr, 'clip_p')
gNeg = Graph(gNeg, prune=True)

gNeg = filterBySign(gNeg, 'clip_c', positive = False)
gNeg = Graph(gNeg, prune=True)
gNeg.ep.clip_c.a = np.abs(gNeg.ep.clip_c.a)

bs = blocks['head']

state = NestedBlockState(gNeg, bs=bs,
                        state_args=dict(recs=[gNeg.ep.clip_c],
                                        rec_types=["real-normal"]))
state.g

labelVertices(state)
# gNeg.vp.level_0 = gNeg.new_vertex_property("double", state.get_bs()[0])
# first = first_occurrence(np.array([gNeg.vp.level_0.a]))
# gNeg.vp.labels = gNeg.new_vp("string", [str(int(x)) if i in first else "" for i, x in enumerate(gNeg.vp.level_0.a)])

<NestedBlockState object, with base <BlockState object with 282 blocks (282 nonempty), degree-corrected, with 1 edge covariate, for graph <Graph object, undirected, with 6826 vertices and 3480 edges, 4 internal vertex properties, 7 internal edge properties, at 0x7f0e4e3dc810>, at 0x7f0e50170890>, and 6 levels of sizes [(6826, 282), (282, 67), (67, 16), (16, 4), (4, 2), (2, 2)] at 0x7f0ddc89c9d0>

In [47]:
state.g.vp.labels

<VertexPropertyMap object with value type 'string', for Graph 0x7f0e4e3dc810, at 0x7f0dbcaa3dd0>

In [49]:

state = NestedBlockState(gNeg, bs=bs,
                        state_args=dict(recs=[gNeg.ep.clip_c],
                                        rec_types=["real-normal"]))
labelVertices(state)
pos, t, tpos = state.draw(eorder=state.g.ep.clip_c,
                        edge_color=prop_to_size(state.g.ep.clip_c, mi=0, ma=1, power=1, log=False),
                        ecmap=(mpl.inferno, .6), 
                        edge_gradient=[], 
                        vertex_size = 10,
                        vertex_text = state.g.vp.labels,
                        vertex_text_position='centered',
                        hvertex_size = 25,
                        hvertex_text = state.g.vp.labels,
                        hedge_pen_width = 3,
                        subsample_edges = 1000, 
                        output = "../../tmp/test.png", 
                        output_size=(2000, 2000))